<a href="https://colab.research.google.com/github/iisabelagarcia/jogoDaVelhaComIA/blob/main/jogoDaVelhaComIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ipywidgets import Button, Layout, VBox, HBox
from functools import partial
import numpy as np


class Interface:

    def __init__(self):
        self.board = np.zeros(shape = (3, 3))
        self.buttons = [
            Button(description='',
                layout=Layout(height='100px', width='100px'))
            for _ in range(9)
        ]

        self.board_interface = VBox([
            HBox(self.buttons[3*i: (3*i)+3])
            for i in range(3)
        ])
        self.user = 'X'
        self.link_clicks()

    def set_position(self, idx, button):
        print(f"Clicked on {idx}")
        if not button.description:
            button.description = self.user
            self.user = 'O' if self.user == 'X' else 'X'

    def link_clicks(self):
        for i, button in enumerate(self.buttons):
            button.on_click(partial(self.set_position, i))


interface = Interface()
interface.board_interface


Clicked on 0
Clicked on 1
Clicked on 2
Clicked on 3
Clicked on 4
Clicked on 5
Clicked on 6
Clicked on 7
Clicked on 8


In [ ]:
def blank(board):
        indices = []
        for i, row in enumerate(board):
            for j, collumn in enumerate(row):
                if collumn == 0:
                    indices.append([i,j])
        return indices

def moves(board, player):
    possibilities = []
    empty_spaces = blank(board)
    for row, col in empty_spaces:
        new_board = board.copy()
        new_board[row][col] = player
        possibilities.append(new_board)

    return possibilities


board = np.zeros((3,3))
moves(board, 1)

[array([[1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 1., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 0.],
        [1., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 0.],
        [0., 1., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 0.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [1., 0., 0.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 1., 0.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 1.]])]

In [ ]:
def game_status(board):
    """X = +1, O = -1, '0' """

    for row in board:
        sum_ = 0
        for item in row:
            sum_ += item

        if sum_ == 3 or sum_ == -3:
            return True, sum_ // 3

    for column in board.T:
        sum_ = 0
        for item in column:
            sum_ += item

        if sum_ == 3 or sum_ == -3:
            return True, sum_ // 3

    #Diagonal principal
    diagonal_1 = np.diagonal(board)
    sum_ = 0

    for item in diagonal_1:
        sum_ += item

    if sum_ == 3 or sum_ == -3:
        return True, sum_ // 3

    #Diagonal secundaria
    diagonal_2 = np.diagonal(np.fliplr(board))
    sum_ = 0

    for item in diagonal_2:
        sum_ += item

    if sum_ == 3 or sum_ == -3:
        return True, sum_ // 3

    if not blank(board):
        return True, None

    return False, None


board = np.zeros((3,3))
game_status(board)

(False, None)

In [ ]:
def fatorial(n):
    return n * fatorial(n-1) if n > 1 else 1

fatorial(6)

720

In [ ]:
def prepare_inputs(board):
    return tuple(map(tuple, map(tuple, board)))

prepare_inputs(np.zeros((3,3)))

((0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0))

In [ ]:
known_nodes = {}
def greedy(function):
    def wrapper(board, player, *args, **kwargs):
        board_input = prepare_inputs(board)
        if (board_input, player) in known_nodes:
            return known_nodes[(board_input, player)]

        output = function(board, player, *args, **kwargs)
        known_nodes[(board_input, player)] = output

        return output

    return wrapper

In [ ]:
def get_score(depth, winner):
    score = 0
    if winner is None:
        return 0

    if winner == 1:
        return 10 - depth

    if winner == -1:
        return depth - 10

@greedy
def minmax(board, player=1, depth=0):
    is_end, winner = game_status(board)

    if is_end:
        return get_score(depth, winner), board

    possibilities = moves(board, player)
    new_player = player * -1

    best_move = None
    best_score = -np.infty if player == 1 else np.infty

    for new_board in possibilities:
        score, _ = minmax(new_board, new_player, depth + 1)
        if player == 1 and score > best_score:  # max
            best_score = score
            best_move = new_board

        if player == -1 and score < best_score:  # min
            best_score = score
            best_move = new_board

    return best_score, best_move

known_nodes = {}
board = np.zeros((3, 3))
minmax(board, 1, 0)

(0,
 array([[1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]))

In [ ]:
MESSAGES = {
    +1: 'IA ganhou!',
    -1: 'Você ganhou!',
    None: 'Empate!'
}

class AIInterface(Interface):
    def __init__(self, ai_starts=False):
        self.board = np.zeros(shape = (3, 3))
        super().__init__()
        if ai_starts:
            score, board = minmax(self.board, player=1)
            self.board = np.array(board)
            self.update()

    def set_position(self, idx, button):
        if self.board[idx // 3, idx % 3] == 0:
            self.board[idx // 3, idx % 3] = -1
            score, board = minmax(self.board, player=1)
            self.board = np.array(board)

            is_over, winner = game_status(self.board)
            if is_over:
                for button in self.buttons:
                    button.disabled = True

                print(MESSAGES.get(winner))

            self.update()

    def update(self):
        for i, row in enumerate(self.board):
            for j, item in enumerate(row):
                self.buttons[3 * i + j].description = ('X' if item == +1
                                                       else ('O' if item == -1 else ''))

interface = AIInterface(ai_starts=False)
interface.board_interface

IA ganhou!
